In [1]:
import pandas as pd
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestClassifier
import catboost as cat
from catboost import CatBoostClassifier
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import log_loss, confusion_matrix, accuracy_score
import xgboost as xgb
import lightgbm as lgb
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [34]:
#importing files
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample = pd.read_csv('sample_submission.csv')

In [3]:
def fill_missing_values(data):
    
    '''
    Function to input missing values based on the column object type
    '''
    
    cols = list(data.columns)
    for col in cols:
        if data[col].dtype == 'int64' or data[col].dtype == 'float64':
        
            data[col] = data[col].fillna(data[col].mean())
        
        #elif data[col].dtype == 'O' or data[col].dtype == 'object':
        #    data[col] = data[col].fillna(data[col].mode()[0])
            
        else:
            data[col] = data[col].fillna(data[col].mode()[0])
            
    return data

In [4]:
def freq_encoding(traindata, testdata, *args):
    
    for _ in args:
        train_enc = (traindata.groupby(_).size()) / len(traindata)
        test_enc = (testdata.groupby(_).size()) / len(testdata)
        
        traindata[_] = traindata[_].apply(lambda x: train_enc[x])
        testdata[_] = testdata[_].apply(lambda x: test_enc[x])
    
    return traindata, testdata

In [5]:
def one_hot_encoding(traindata, testdata, *args):
    
    for ii in args:
        traindata = pd.get_dummies(traindata, prefix=[ii], columns=[ii])
        testdata = pd.get_dummies(testdata, prefix=[ii], columns=[ii])
        
    return traindata, testdata

In [6]:
def drop_columns(traindata, testdata, target, *args):
    
    labels = np.array(traindata[target])
    
    columns = []
    for _ in args:
        columns.append(_)
        
    traindata = traindata.drop(columns, axis=1)
    traindata = traindata.drop(target, axis=1)
    testdata = testdata.drop(columns, axis=1)
        
    return labels, traindata, testdata

In [7]:
def mean_target(data, cols):
    kf = KFold(5)
    a = pd.DataFrame()
    for tr_ind, val_ind in kf.split(data):
        X_tr, X_val= data.iloc[tr_ind].copy(), data.iloc[val_ind].copy()
        for col in cols:
            means = X_val[col].map(X_tr.groupby(col).CHURN.mean())
            X_val[col + '_mean_target'] = means + 0.0001
        a = pd.concat((a, X_val))
    prior = CHURN.mean()
    a.fillna(prior, inplace=True)
    return a

In [8]:
def process(traindata, testdata):
    
    cols = list(traindata.columns)
    for _ in cols:
        traindata[_] = np.where(traindata[_] == np.inf, 0, traindata[_])
        testdata[_] = np.where(testdata[_] == np.inf, 0, testdata[_])
        traindata[_] = np.where(traindata[_] == np.nan, 0, traindata[_])
        testdata[_] = np.where(testdata[_] == np.nan, 0, testdata[_])
        traindata[_] = np.where(traindata[_] == -np.inf, 0, traindata[_])
        testdata[_] = np.where(testdata[_] == -np.inf, 0, testdata[_])
        
    return traindata, testdata

In [60]:
ntrain = train.shape[0]
ntest = test.shape[0]
target = train.CHURN.copy()
df = pd.concat((train, test)).reset_index(drop=True)
df = df.drop('user_id', 1)

In [61]:
cols = ['ZONE1', 'ZONE2', 'TIGO', 'MRG']

labels, train1, test1 = drop_columns(df, test, 'CHURN', *cols)

one_hot_cols = ['REGION', 'TENURE']

train1, test1 = one_hot_encoding(train1, test1, *one_hot_cols)

In [62]:
train1.shape

(500000, 33)

In [63]:
train1 = train1.fillna(-999)
test1 = test1.fillna(-999)

In [15]:
freq_col = ['TOP_PACK']

train1, test1 = freq_encoding(train1, test1, *freq_col)

In [64]:
train1['REG*REV'] = train1['REGULARITY'] * train1['REVENUE']
train1['REG/REV'] = train1['REGULARITY'] / train1['REVENUE']
train1['LOG_REV'] = np.log(train1['REVENUE'])
train1['REG*MUT'] = train1['REGULARITY'] * train1['MONTANT']
train1['REG/MUT'] = train1['REGULARITY'] / train1['MONTANT']
train1['LOG_MUT'] = np.log(train1['MONTANT'])
train1['REG*NET'] = train1['REGULARITY'] * train1['ON_NET']
train1['REG/NET'] = train1['REGULARITY'] / train1['ON_NET']
train1['FREQ+FREQ'] = train1['FREQUENCE_RECH'] + train1['FREQUENCE']
train1['FREQ-FREQ'] = train1['FREQUENCE_RECH'] - train1['FREQUENCE']
train1['LOG_NET'] = np.log(train1['ON_NET'])
train1['REV+DAT'] = train1['REVENUE'] + train1['DATA_VOLUME']
train1['REV-DAT'] = train1['REVENUE'] - train1['DATA_VOLUME']
train1['REV-DAT'] = train1['REVENUE'] - train1['DATA_VOLUME']
train1['REV/DAT'] = train1['REVENUE'] / train1['DATA_VOLUME']
train1['REV-ARP'] = train1['REVENUE'] - train1['ARPU_SEGMENT']
train1['REV+ARP'] = train1['REVENUE'] + train1['ARPU_SEGMENT']
train1['REV*ARP'] = train1['REVENUE'] * train1['ARPU_SEGMENT']
train1['REV/ARP'] = train1['REVENUE'] / train1['ARPU_SEGMENT']
train1['REV*FREQ'] = train1['REVENUE'] * train1['FREQUENCE']
train1['REV/FREQ'] = train1['REVENUE'] / train1['FREQUENCE']
train1['LOG_DAT'] = np.log(train1['DATA_VOLUME'])
train1['LOG_ARP'] = np.log(train1['ARPU_SEGMENT'])

test1['REG*REV'] = test1['REGULARITY'] * test1['REVENUE']
test1['REG/REV'] = test1['REGULARITY'] / test1['REVENUE']
test1['LOG_REV'] = np.log(test1['REVENUE'])
test1['REG*MUT'] = test1['REGULARITY'] * test1['MONTANT']
test1['REG/MUT'] = test1['REGULARITY'] / test1['MONTANT']
test1['LOG_MUT'] = np.log(test1['MONTANT'])
test1['REG*NET'] = test1['REGULARITY'] * test1['ON_NET']
test1['REG/NET'] = test1['REGULARITY'] / test1['ON_NET']
test1['FREQ+FREQ'] = test1['FREQUENCE_RECH'] + test1['FREQUENCE']
test1['FREQ-FREQ'] = test1['FREQUENCE_RECH'] - test1['FREQUENCE']
test1['LOG_NET'] = np.log(test1['ON_NET'])
test1['REV+DAT'] = test1['REVENUE'] + test1['DATA_VOLUME']
test1['REV-DAT'] = test1['REVENUE'] - test1['DATA_VOLUME']
test1['REV-DAT'] = test1['REVENUE'] - test1['DATA_VOLUME']
test1['REV/DAT'] = test1['REVENUE'] / test1['DATA_VOLUME']
test1['REV-ARP'] = test1['REVENUE'] - test1['ARPU_SEGMENT']
test1['REV+ARP'] = test1['REVENUE'] + test1['ARPU_SEGMENT']
test1['REV*ARP'] = test1['REVENUE'] * test1['ARPU_SEGMENT']
test1['REV/ARP'] = test1['REVENUE'] / test1['ARPU_SEGMENT']
test1['REV*FREQ'] = test1['REVENUE'] * test1['FREQUENCE']
test1['REV/FREQ'] = test1['REVENUE'] / test1['FREQUENCE']
test1['LOG_DAT'] = np.log(test1['DATA_VOLUME'])
test1['LOG_ARP'] = np.log(test1['ARPU_SEGMENT'])

In [65]:
train1, test1 = process(train1, test1)

In [66]:
train1 = train1.fillna(-999)
test1 = test1.fillna(-999)

In [67]:
CHURN = train.CHURN.copy()

train2['CHURN'] = labels
train1['CHURN'] = labels
train2 = train1.copy()
train2 = mean_target(train1, ['TOP_PACK'])

In [68]:
train3 = train2[:ntrain].copy()
CHURN = train3.CHURN.copy()
train3.drop('CHURN', axis=1, inplace=True)

test3 = train2[ntrain:].copy()
test3.drop('CHURN', axis=1, inplace=True)
test3 = test3.reset_index(drop=True)

In [69]:
train3.shape, test3.shape

((400000, 56), (100000, 56))